# Analyser data fra kompensasjonsordningen med Python og Pandas

I denne notatboken brukes data om tildelinger av støtte fra Kompensasjonsordningen for næringslivet.
Dataene ligger som både .json og .csv i dette repositoriet, men inneholder ikke de siste tallene. Dersom du ønsker å bruke oppdaterte tall kan de hentes fra hhv [link1] og [link2].

Steg 0: Installer Pandas

Dersom du ikke har gjort det allerede, installer Pandas. Bruk % for å indikere at det ikke er python-kode men en 'magisk' kommando:

In [1]:
%pip install pandas

     |████████████████████████████████| 9.7 MB 3.2 MB/s 
     |████████████████████████████████| 15.4 MB 19.3 MB/s 
     |████████████████████████████████| 510 kB 13.2 MB/s 
Note: you may need to restart the kernel to use updated packages.


Steg 1: Litt info om datasettet

Vi starter med å se på csv-filen, og finne litt ut om den

Vedtak skjer løpende. Oppdaterte data legges ut hver time. For de nyeste dataene gjør ...

In [7]:
import pandas as pd

df = pd.read_csv('tilskudd.csv', sep=';')
df.head()


,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,naeringskode1,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
0,2021009917,913686454,AKTIV365 SANDSLI AS,4601,BERGEN,4600.0,Vestland,93.130,NaN,NaN,"125488,00",2021-03-08T00:00,november-desember2020,november-desember2019,"327907,57","766608,00","1394409,00",0.85,988782041,COLLEGIUM REVISJON AS
1,2021009916,988237612,GOLDER ASSOCIATES AS (NORWAY),3005,DRAMMEN,3000.0,Viken,71.129,NaN,NaN,"208706,41",2021-03-08T00:00,november-desember2020,november-desember2019,"554372,00","5147270,00","9239568,00",0.85,993606650,BDO AS
2,2021009915,967265403,ANITA GULBRANDSEN LØKEN,3020,NORDRE FOLLO,3000.0,Viken,49.320,NaN,NaN,"15008,28",2021-03-08T00:00,november-desember2020,november-desember2019,"34995,00","196945,00","397509,00",0.85,922829845,PARTNER REVISJON AS
3,2021009914,918718761,THAI AROMA MASSASJE AS,301,OSLO,300.0,Oslo,96.040,NaN,NaN,"7950,66",2021-03-08T00:00,november-desember2020,november-desember2019,"24905,00","81300,00","130200,00",0.85,913423585,MYHRERS RTB AS
4,2021009913,919965614,GLH LADE AS,5001,TRONDHEIM,5000.0,Trøndelag Trööndelage,47.599,NaN,NaN,"83134,37",2021-03-08T00:00,september-oktober2020,september-oktober2019,"287994,00","2328281,00","3962232,00",0.70,989271733,VEKTE AS


Vi ser at ... osv ...

In [9]:
df.describe()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,kommunenummer,fylkesnummer,naeringskode1,naeringskode2,naeringskode3,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer
count,9.048000e+03,9.048000e+03,9048.000000,9012.000000,9048.000000,244.000000,9.000000,9048.000000,9.048000e+03
mean,2.021005e+09,9.481143e+08,2662.386273,2651.364847,58.102011,54.737045,48.966889,0.789838,9.544368e+08
std,2.847589e+03,4.406056e+07,1772.681531,1772.489596,17.751790,22.596203,29.426245,0.073522,4.161825e+07
min,2.021000e+09,8.116188e+08,301.000000,300.000000,0.000000,0.000000,10.710000,0.700000,8.121884e+08
25%,2.021003e+09,9.168709e+08,1103.000000,1100.000000,49.320000,47.285250,15.200000,0.700000,9.206667e+08
50%,2.021005e+09,9.579039e+08,3024.000000,3000.000000,56.101000,55.202000,55.202000,0.850000,9.744623e+08
75%,2.021007e+09,9.880344e+08,4601.000000,4600.000000,70.220000,70.100000,78.200000,0.850000,9.875649e+08
max,2.021010e+09,9.996654e+08,5444.000000,5400.000000,96.090000,96.020000,79.909000,0.850000,9.996475e+08


Det som er mest interessant å få frem verdier på er ikke med, som tilskuddsbeløp, og ulike omsetningstall. Det skyldes at disse verdiene leses inn som tekst istedenfor tall, pga komma som desimaltegn.


In [10]:
df.columns

Index(['tilskuddsnummer', 'tilskuddsmottakerOrganisasjonsnummer',
       'tilskuddsmottakerNavn', 'kommunenummer', 'kommune', 'fylkesnummer',
       'fylke', 'naeringskode1', 'naeringskode2', 'naeringskode3',
       'tilskuddsbelop', 'tildelingsdato', 'tilskuddsperiode',
       'sammenligningsperiode',
       'sumFasteUungaaeligeKostanderITilskuddsperioden',
       'omsetningITilskuddsperiode', 'omsetningISammenligningsperiode',
       'justeringsfaktor', 'uavhengigMedsignererOrganisasjonsnummer',
       'uavhengigMedsignererNavn'],
      dtype='object')

In [18]:
df['tilskuddsbelop']


0       125488,00
1       208706,41
2        15008,28
3         7950,66
4        83134,37
          ...    
9043     20367,53
9044     56709,39
9045      9844,00
9046     11624,90
9047    156339,75
Name: tilskuddsbelop, Length: 9048, dtype: object

Må lese inn Excel-fila igjen og denne gangen presiserer vi hva som er desimalskilletegnet:

In [19]:
df = pd.read_csv('tilskudd.csv', sep=';', decimal=',')


In [20]:
df.describe()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,kommunenummer,fylkesnummer,tilskuddsbelop,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,uavhengigMedsignererOrganisasjonsnummer
count,9.048000e+03,9.048000e+03,9048.000000,9012.000000,9.048000e+03,9.048000e+03,9.048000e+03,9.048000e+03,9.048000e+03
mean,2.021005e+09,9.481143e+08,2662.386273,2651.364847,1.910668e+05,4.228766e+05,1.433093e+06,1.424610e+07,9.544368e+08
std,2.847589e+03,4.406056e+07,1772.681531,1772.489596,2.037838e+06,4.467720e+06,8.715873e+06,1.013714e+09,4.161825e+07
min,2.021000e+09,8.116188e+08,301.000000,300.000000,5.000340e+03,6.410000e+03,0.000000e+00,6.000000e+03,8.121884e+08
25%,2.021003e+09,9.168709e+08,1103.000000,1100.000000,1.718204e+04,4.087875e+04,1.117474e+05,3.516660e+05,9.206667e+08
50%,2.021005e+09,9.579039e+08,3024.000000,3000.000000,4.043306e+04,9.759950e+04,3.093570e+05,9.054335e+05,9.744623e+08
75%,2.021007e+09,9.880344e+08,4601.000000,4600.000000,1.023590e+05,2.405640e+05,9.022215e+05,2.425666e+06,9.875649e+08
max,2.021010e+09,9.996654e+08,5444.000000,5400.000000,8.188874e+07,1.918143e+08,4.146793e+08,9.640463e+10,9.996475e+08


In [21]:
df['tilskuddsbelop']

0       125488.00
1       208706.41
2        15008.28
3         7950.66
4        83134.37
          ...    
9043     20367.53
9044     56709.39
9045      9844.00
9046     11624.90
9047    156339.75
Name: tilskuddsbelop, Length: 9048, dtype: float64